In [1]:
import pandas as pd
import re
import ast
from fuzzywuzzy import fuzz, process
import Levenshtein

pd.set_option('max_columns', None)
pd.options.display.max_colwidth = 1000

In [2]:
df_lecturas = pd.read_csv('../data/processed/recipe_df_lecturas.csv')

df_recetasgratis = pd.read_csv('../data/processed/recipe_df_recetasgratis.csv')

In [3]:
recipe_df = pd.concat([df_lecturas, df_recetasgratis], ignore_index=True)

In [4]:
recipe_df.head(1)

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,main_category,num_ingredients
0,https://content-recetas.lecturas.com/medio/2019/12/16/ensalada_nicoise_y_ensalada_del_perigord_c0d6d2e1_800x800.jpg,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nicoise_8575.html,"['patatas pequeñas rojas', 'huevos', 'judías verdes', 'tomates maduros', 'atún en conserva', 'espinacas baby', 'cebolla morada', 'perejil fresco picado', 'aceite de oliva virgen extra', 'limón el zumo', 'pimienta', 'sal']",60.0,0,0,0,0,0,principal,12


In [5]:
def ingred_list(column):
    ingred_list = ast.literal_eval(column)
    return ingred_list

In [6]:
recipe_df["ingredients_list"] = recipe_df.apply(lambda x: ingred_list(x["ingredients_list"]), axis=1)

In [7]:
def clean_ingredients(column):
    new_list = []
    for i in column:
        try:
            huevo_ratio = fuzz.WRatio("huevo", i)
            if huevo_ratio >= 86:
                new_list.append("huevo")
            else:
                new_list.append(i)
        except:
            new_list.append(i)
    return new_list

In [8]:
def clean_ingredients_2(ingred_column):
    #ingred = ast.literal_eval(ingred_column)
    ingredients_list = []
    for ing in ingred_column:
        clean_ingred = re.sub(" ?unas hebras de |un tarro de |unas gotas de |un chorro de |lata |tzas |taza |cuchara sopera |c.c |una tacita de | ?mitad será ?| que se encuentre en la página| la necesaria|cucharadas |cdas | a temperatura ambiente|un poco de |cuadritos |una barra y cuarto de |cuadritos de |tbls ?|2 tipo cucharillas fritas ?|una nuez de |ud. ?| cortad[oa][s]? en.*| median[oa][s]?|  pequeñ[oa][s]?|  grand[s]?", "", ing)
        clean_ingred2 = re.sub("\d","", clean_ingred)
        ingredients_list.append(clean_ingred2)
    ingredients_list = [i for i in ingredients_list if i]
    return ingredients_list

In [9]:
recipe_df["ingredients_list"] = recipe_df.apply(lambda x: clean_ingredients(x["ingredients_list"]), axis=1)

In [10]:
recipe_df["ingredients_list"] = recipe_df.apply(lambda x: clean_ingredients_2(x["ingredients_list"]), axis=1)

In [11]:
recipe_df

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,main_category,num_ingredients
0,https://content-recetas.lecturas.com/medio/2019/12/16/ensalada_nicoise_y_ensalada_del_perigord_c0d6d2e1_800x800.jpg,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nicoise_8575.html,"[patatas pequeñas rojas, huevo, jas verdes, tomates maduros, atún en conserva, espinacas baby, cebolla morada, perejil fresco picado, aceite de oliva virgen extra, limón el zumo, pimienta, sal]",60.0,0,0,0,0,0,principal,12
1,https://content-recetas.lecturas.com/medio/2019/07/17/ensalada_nicoise_y_ensalada_del_perigord_8e285865_800x800.jpg,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-perigord_8576.html,"[foie micuit, beicon gruesas, champiñones, berros, rúcula, tomates secos en aceite, piñones, jamón de pato, aceite de oliva]",25.0,0,0,0,0,0,principal,9
2,https://content-recetas.lecturas.com/medio/2020/03/05/xato_con_salsa_romesco_y_salpicon_de_marisco_d6b2df39_800x800.jpg,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-romesco_8577.html,"[bacalao desalado, cebolleta, escarolas, aceitunas negras, anchoas en aceite, tomates maduros, cabeza de ajos, ñoras, pan seco, almendras peladas tostadas, avellanas peladas y tostadas, pimentón, sal, aceite de oliva, vinagre de vino blanco, guindillas]",35.0,0,0,0,0,0,principal,16
3,https://content-recetas.lecturas.com/medio/2019/07/25/ensalada_de_ceviche_de_langostinos_y_cebolla_y_ensalada_de_pollo_remolacha_y_manzana_ca9d87d5_800x800.jpg,Ensalada de ceviche de langostinos y cebolla,https://www.lecturas.com/recetas/ensalada-ceviche-langostinos-y-cebolla_8447.html,"[langostinos, pimienta, limas, tomates, cebolla morada, limones, guindilla, aguacate, cilantro, sal, lechugas brotes, germinados de remolacha]",25.0,0,0,0,0,0,principal,12
4,https://content-recetas.lecturas.com/medio/2019/08/26/ensalada_de_ceviche_de_langostinos_y_cebolla_y_ensalada_de_pollo_remolacha_y_manzana_bf72acd4_800x800.jpg,"Ensalada de pollo, remolacha y manzana",https://www.lecturas.com/recetas/ensalada-pollo-remolacha-y-manzana_8448.html,"[pechugas de pollo, manzanas ácida, col lombarda, zanahorias, granada, remolacha, nueces peladas, pan rallado grueso, pimentón dulce, huevo, comino, semillas de sésamo negro, lechugas, aceite de oliva, sal, pimienta]",40.0,0,0,0,0,0,principal,16
...,...,...,...,...,...,...,...,...,...,...,...,...
18235,https://t2.rg.ltmcdn.com/es/images/8/0/0/fritos_de_acelga_47008_600.jpg,Fritos de acelga,https://www.recetasgratis.net/receta-de-fritos-de-acelga-47008.html,"[acelgas sin tallos, harina , leche , hornear, cebolla frita, huevo, sal, aceite , papel absorbente]",30.0,0,0,0,0,0,principal,9
18236,https://t1.rg.ltmcdn.com/es/images/2/6/8/img_verduras_salteadas_en_mantequilla_46862_600.jpg,Verduras salteadas en mantequilla,https://www.recetasgratis.net/receta-de-verduras-salteadas-en-mantequilla-46862.html,"[zapallo italiano, pimentón rojo o verde, champiñón, zanahoria, coliflor en ramas pequeñas, brócoli en ramas pequeñas, mantequilla o aceite de oliva, sal, pimienta]",30.0,0,0,0,0,0,principal,9
18237,https://t1.rg.ltmcdn.com/es/images/0/8/6/img_guisantes_compuestos_con_thermomix_46680_600.jpg,Guisantes compuestos con Thermomix,https://www.recetasgratis.net/receta-de-guisantes-compuestos-con-thermomix-46680.html,"[cebolla, aceite, guisantes congelados, unos trocitos de jamón serrano en tacos, huevo, pimentón, doble caldo]",45.0,0,0,0,0,0,principal,7
18238,https://t2.rg.ltmcdn.com/es/images/4/6/6/papas_a_la_crema_con_champinones_46664_600.jpg,Papas a la crema con champiñones,https://www.recetasgratis.net/receta-de-papas-a-la-crema-con-champinones-46664.html,"[papas fritas grande lisas, crema, queso laminado y/o rallado, champiñón limpios, margarina, sal, pimienta]",30.0,0,0,0,0,0,principal,7


In [12]:
recipe_df["gluten_free"].value_counts()

0    16621
1     1619
Name: gluten_free, dtype: int64

In [13]:
recipe_df["egg_free"].value_counts()

0    18136
1      104
Name: egg_free, dtype: int64

In [14]:
recipe_df["sucrose_fructose_free"].value_counts()

0    17968
1      272
Name: sucrose_fructose_free, dtype: int64

In [15]:
recipe_df["low_sodium"].value_counts()

0    18092
1      148
Name: low_sodium, dtype: int64

In [16]:
recipe_df["lactose_free"].value_counts()

0    16695
1     1545
Name: lactose_free, dtype: int64

In [17]:
recipe_df.to_csv (f'../data/processed/recipe_df_demo.csv', index=False)

In [18]:
recipe_df.columns

Index(['recipe_image_url', 'recipe_name', 'recipe_url', 'ingredients_list',
       'time_preparation(min)', 'gluten_free', 'egg_free',
       'sucrose_fructose_free', 'low_sodium', 'lactose_free', 'main_category',
       'num_ingredients'],
      dtype='object')

In [19]:
ingredients_df = recipe_df.ingredients_list.apply(pd.Series) \
    .merge(recipe_df, right_index = True, left_index = True) \
    .melt(id_vars = ['recipe_image_url', 'recipe_name', 'recipe_url', 'ingredients_list',
       'time_preparation(min)', 'gluten_free', 'egg_free',
       'sucrose_fructose_free', 'low_sodium', 'lactose_free', 'main_category',
       'num_ingredients'], value_name = "ingredient")

In [20]:
ingredients_df = ingredients_df[ingredients_df['ingredient'].notna()]

In [21]:
ingredients_df.head()

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,main_category,num_ingredients,variable,ingredient
0,https://content-recetas.lecturas.com/medio/2019/12/16/ensalada_nicoise_y_ensalada_del_perigord_c0d6d2e1_800x800.jpg,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nicoise_8575.html,"[patatas pequeñas rojas, huevo, jas verdes, tomates maduros, atún en conserva, espinacas baby, cebolla morada, perejil fresco picado, aceite de oliva virgen extra, limón el zumo, pimienta, sal]",60.0,0,0,0,0,0,principal,12,0,patatas pequeñas rojas
1,https://content-recetas.lecturas.com/medio/2019/07/17/ensalada_nicoise_y_ensalada_del_perigord_8e285865_800x800.jpg,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-perigord_8576.html,"[foie micuit, beicon gruesas, champiñones, berros, rúcula, tomates secos en aceite, piñones, jamón de pato, aceite de oliva]",25.0,0,0,0,0,0,principal,9,0,foie micuit
2,https://content-recetas.lecturas.com/medio/2020/03/05/xato_con_salsa_romesco_y_salpicon_de_marisco_d6b2df39_800x800.jpg,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-romesco_8577.html,"[bacalao desalado, cebolleta, escarolas, aceitunas negras, anchoas en aceite, tomates maduros, cabeza de ajos, ñoras, pan seco, almendras peladas tostadas, avellanas peladas y tostadas, pimentón, sal, aceite de oliva, vinagre de vino blanco, guindillas]",35.0,0,0,0,0,0,principal,16,0,bacalao desalado
3,https://content-recetas.lecturas.com/medio/2019/07/25/ensalada_de_ceviche_de_langostinos_y_cebolla_y_ensalada_de_pollo_remolacha_y_manzana_ca9d87d5_800x800.jpg,Ensalada de ceviche de langostinos y cebolla,https://www.lecturas.com/recetas/ensalada-ceviche-langostinos-y-cebolla_8447.html,"[langostinos, pimienta, limas, tomates, cebolla morada, limones, guindilla, aguacate, cilantro, sal, lechugas brotes, germinados de remolacha]",25.0,0,0,0,0,0,principal,12,0,langostinos
4,https://content-recetas.lecturas.com/medio/2019/08/26/ensalada_de_ceviche_de_langostinos_y_cebolla_y_ensalada_de_pollo_remolacha_y_manzana_bf72acd4_800x800.jpg,"Ensalada de pollo, remolacha y manzana",https://www.lecturas.com/recetas/ensalada-pollo-remolacha-y-manzana_8448.html,"[pechugas de pollo, manzanas ácida, col lombarda, zanahorias, granada, remolacha, nueces peladas, pan rallado grueso, pimentón dulce, huevo, comino, semillas de sésamo negro, lechugas, aceite de oliva, sal, pimienta]",40.0,0,0,0,0,0,principal,16,0,pechugas de pollo


In [22]:
ingredients_df.to_csv (f'../data/processed/ingredients_df.csv', index=False)

In [23]:
input_ing = input('¿Qué tienes hoy en la nevera?')

¿Qué tienes hoy en la nevera? leche, huevos, jamón, aceite de oliva, sal, chorizo, guisantes, zanahorias, tomate


In [24]:
input_ing

'leche, huevos, jamón, aceite de oliva, sal, chorizo, guisantes, zanahorias, tomate'

In [25]:
def input_list(string):
    input_list = []
    str_split = string.split(", ")
    for x in str_split:
        input_list.append(x)
    return input_list

input_ingred = input_list(input_ing)

print(input_ingred)

['leche', 'huevos', 'jamón', 'aceite de oliva', 'sal', 'chorizo', 'guisantes', 'zanahorias', 'tomate']


In [26]:
recipe_df["input"] = input_ing
recipe_df["input"] = recipe_df.apply(lambda x: input_list(x["input"]), axis=1)

In [27]:
def matched(ingred_list, input_ingred):
    new_list = []
    for element in input_ingred:
        try:
            best_match = process.extract(element, ingred_list, scorer=fuzz.WRatio, limit=1)
            if element == "sal" or element == "harina" or element == "azúcar" or element == "pimienta":
                if best_match[0][1] >= 90:
                    ingred_list.remove(best_match[0][0])
                    new_list.append(best_match[0][0])
            elif element == "harina":
                if best_match[0][1] >= 90:
                    ingred_list.remove(best_match[0][0])
                    new_list.append(best_match[0][0])
            elif element == "azúcar":
                 if best_match[0][1] >= 90:
                    ingred_list.remove(best_match[0][0])
                    new_list.append(best_match[0][0])
            elif element == "pimienta":
                if best_match[0][1] >= 90:
                    ingred_list.remove(best_match[0][0])
                    new_list.append(best_match[0][0])
            else:
                if best_match[0][1] >= 86:
                    ingred_list.remove(best_match[0][0])
                    new_list.append(best_match[0][0])
        except:
            pass
    return new_list

In [28]:
recipe_df["matched_ingred"] = recipe_df.apply(lambda x: matched(x["ingredients_list"], x["input"])
                                                  , axis=1)

In [29]:
recipe_df.head()

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,main_category,num_ingredients,input,matched_ingred
0,https://content-recetas.lecturas.com/medio/2019/12/16/ensalada_nicoise_y_ensalada_del_perigord_c0d6d2e1_800x800.jpg,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nicoise_8575.html,"[patatas pequeñas rojas, jas verdes, atún en conserva, espinacas baby, cebolla morada, perejil fresco picado, limón el zumo, pimienta]",60.0,0,0,0,0,0,principal,12,"[leche, huevos, jamón, aceite de oliva, sal, chorizo, guisantes, zanahorias, tomate]","[huevo, aceite de oliva virgen extra, sal, tomates maduros]"
1,https://content-recetas.lecturas.com/medio/2019/07/17/ensalada_nicoise_y_ensalada_del_perigord_8e285865_800x800.jpg,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-perigord_8576.html,"[foie micuit, beicon gruesas, champiñones, berros, rúcula, piñones]",25.0,0,0,0,0,0,principal,9,"[leche, huevos, jamón, aceite de oliva, sal, chorizo, guisantes, zanahorias, tomate]","[jamón de pato, aceite de oliva, tomates secos en aceite]"
2,https://content-recetas.lecturas.com/medio/2020/03/05/xato_con_salsa_romesco_y_salpicon_de_marisco_d6b2df39_800x800.jpg,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-romesco_8577.html,"[bacalao desalado, cebolleta, escarolas, aceitunas negras, anchoas en aceite, cabeza de ajos, ñoras, pan seco, almendras peladas tostadas, avellanas peladas y tostadas, pimentón, vinagre de vino blanco, guindillas]",35.0,0,0,0,0,0,principal,16,"[leche, huevos, jamón, aceite de oliva, sal, chorizo, guisantes, zanahorias, tomate]","[aceite de oliva, sal, tomates maduros]"
3,https://content-recetas.lecturas.com/medio/2019/07/25/ensalada_de_ceviche_de_langostinos_y_cebolla_y_ensalada_de_pollo_remolacha_y_manzana_ca9d87d5_800x800.jpg,Ensalada de ceviche de langostinos y cebolla,https://www.lecturas.com/recetas/ensalada-ceviche-langostinos-y-cebolla_8447.html,"[langostinos, pimienta, limas, cebolla morada, limones, guindilla, aguacate, cilantro, lechugas brotes]",25.0,0,0,0,0,0,principal,12,"[leche, huevos, jamón, aceite de oliva, sal, chorizo, guisantes, zanahorias, tomate]","[germinados de remolacha, sal, tomates]"
4,https://content-recetas.lecturas.com/medio/2019/08/26/ensalada_de_ceviche_de_langostinos_y_cebolla_y_ensalada_de_pollo_remolacha_y_manzana_bf72acd4_800x800.jpg,"Ensalada de pollo, remolacha y manzana",https://www.lecturas.com/recetas/ensalada-pollo-remolacha-y-manzana_8448.html,"[pechugas de pollo, manzanas ácida, col lombarda, granada, remolacha, nueces peladas, pan rallado grueso, pimentón dulce, comino, semillas de sésamo negro, lechugas, pimienta]",40.0,0,0,0,0,0,principal,16,"[leche, huevos, jamón, aceite de oliva, sal, chorizo, guisantes, zanahorias, tomate]","[huevo, aceite de oliva, sal, zanahorias]"


In [30]:
def num_ingred(column):
    return len(column)

In [31]:
recipe_df["num_matched"] = recipe_df.apply(lambda x: num_ingred(x["matched_ingred"]), axis=1)

In [32]:
def num_missing(recipe_ingred, active_ingred):
    missing_ingred = recipe_ingred - active_ingred
    return missing_ingred


recipe_df["num_missing_ingred"] = recipe_df.apply(lambda x: num_missing(x["num_ingredients"],
                                                                        x["num_matched"]),axis=1)

In [33]:
def missing_ing(matched, ingred_list):
    missing_list = []
    for i in ingred_list:
        if i in matched:
            pass
        else:
            missing_list.append(i)
    return missing_list

In [34]:
recipe_df["missing_ingred"] = recipe_df.apply(lambda x: missing_ing(x["matched_ingred"],
                                                                        x["ingredients_list"]),axis=1)

In [35]:
def rating(match, missing):
    result = 0.70*match-0.30*missing
    return result

In [36]:
recipe_df["recipe_rate"] = recipe_df.apply(lambda x: rating(x["num_matched"], x["num_missing_ingred"]),
                                               axis=1)

In [37]:
filter_principal = recipe_df['main_category'] == "principal"
output_1 = recipe_df[filter_principal].sort_values(by=["recipe_rate", "num_missing_ingred"], 
                               ascending=[False, True]).head(10)

In [38]:
output_1.columns

Index(['recipe_image_url', 'recipe_name', 'recipe_url', 'ingredients_list',
       'time_preparation(min)', 'gluten_free', 'egg_free',
       'sucrose_fructose_free', 'low_sodium', 'lactose_free', 'main_category',
       'num_ingredients', 'input', 'matched_ingred', 'num_matched',
       'num_missing_ingred', 'missing_ingred', 'recipe_rate'],
      dtype='object')

In [39]:
output = output_1[['recipe_image_url', 'recipe_name', 'recipe_url', 'ingredients_list',
       'time_preparation(min)', 'gluten_free', 'egg_free',
       'sucrose_fructose_free', 'low_sodium', 'lactose_free', 'main_category',
       'num_ingredients']]

In [43]:
output_1.to_csv (f'../data/processed/df_demo.csv', index=False)

In [41]:
output_2 = recipe_df[filter_principal].sort_values(by=["num_missing_ingred", "recipe_rate"], 
                               ascending=[True, False]).head(10)

In [42]:
output_2

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,main_category,num_ingredients,input,matched_ingred,num_matched,num_missing_ingred,missing_ingred,recipe_rate
11163,https://t2.rg.ltmcdn.com/es/images/1/5/1/huevos_a_la_flamenca_74151_600.jpg,Huevos a la flamenca,https://www.recetasgratis.net/receta-de-huevos-a-la-flamenca-74151.html,[],45.0,0,0,0,0,0,principal,6,"[leche, huevos, jamón, aceite de oliva, sal, chorizo, guisantes, zanahorias, tomate]","[huevo, jamón, aceite de oliva, chorizo, guisantes, tomate frito]",6,0,[],4.2
17481,https://t1.rg.ltmcdn.com/es/images/2/1/5/img_chips_de_zanahoria_al_horno_61512_600.jpg,Chips de zanahoria al horno,https://www.recetasgratis.net/receta-de-chips-de-zanahoria-al-horno-61512.html,[],15.0,0,0,0,0,0,principal,3,"[leche, huevos, jamón, aceite de oliva, sal, chorizo, guisantes, zanahorias, tomate]","[aceite de oliva, sal, zanahorias]",3,0,[],2.1
11325,https://t1.rg.ltmcdn.com/es/images/0/9/9/img_huevos_fritos_sin_aceite_59990_600.jpg,Huevos fritos sin aceite,https://www.recetasgratis.net/receta-de-huevos-fritos-sin-aceite-59990.html,[],10.0,0,0,0,0,0,principal,2,"[leche, huevos, jamón, aceite de oliva, sal, chorizo, guisantes, zanahorias, tomate]","[huevo, sal]",2,0,[],1.4
2048,https://content-recetas.lecturas.com/medio/2018/07/19/brocheta-de-butifarras-variadas_d43913c7_800x800.jpg,Brochetas de butifarras variadas,https://www.lecturas.com/recetas/brochetas-butifarras-variadas_729.html,[],20.0,1,0,0,0,1,principal,1,"[leche, huevos, jamón, aceite de oliva, sal, chorizo, guisantes, zanahorias, tomate]",[butifarra de setas de payés de escalivada],1,0,[],0.7
7790,https://t2.rg.ltmcdn.com/es/images/7/8/1/img_conejo_en_adobo_51187_600.jpg,Conejo en adobo.,https://www.recetasgratis.net/receta-de-conejo-en-adobo-51187.html,[],NaN,0,0,0,0,0,principal,1,"[leche, huevos, jamón, aceite de oliva, sal, chorizo, guisantes, zanahorias, tomate]",[verduras y sal.],1,0,[],0.7
9279,https://t2.rg.ltmcdn.com/es/images/4/2/3/img_salmorrejo_46324_600.jpg,SALMORREJO,https://www.recetasgratis.net/receta-de-salmorrejo-46324.html,[],NaN,0,0,0,0,0,principal,1,"[leche, huevos, jamón, aceite de oliva, sal, chorizo, guisantes, zanahorias, tomate]","[lomo de cerdo ,longaniza ,guisantes,tallos de esparrago,huevos , aceite,ajos,agua, sal y harina.]",1,0,[],0.7
11184,https://t2.rg.ltmcdn.com/es/images/9/9/7/huevo_poche_al_microondas_71799_600.jpg,Huevo poché al microondas,https://www.recetasgratis.net/receta-de-huevo-poche-al-microondas-71799.html,[],10.0,0,0,0,0,0,principal,1,"[leche, huevos, jamón, aceite de oliva, sal, chorizo, guisantes, zanahorias, tomate]",[huevo],1,0,[],0.7
11595,https://t1.rg.ltmcdn.com/es/images/7/9/2/img_gallina_pinta_facil_47297_600.jpg,Gallina Pinta facil,https://www.recetasgratis.net/receta-de-gallina-pinta-facil-47297.html,[],NaN,0,0,0,0,0,principal,1,"[leche, huevos, jamón, aceite de oliva, sal, chorizo, guisantes, zanahorias, tomate]",[frijol-grano de maiz-pollo o carne de res],1,0,[],0.7
12300,https://t2.rg.ltmcdn.com/es/images/2/5/9/img_bogavante_breton_21952_600.jpg,Bogavante bretón,https://www.recetasgratis.net/receta-de-bogavante-breton-21952.html,[],NaN,0,0,0,0,0,principal,1,"[leche, huevos, jamón, aceite de oliva, sal, chorizo, guisantes, zanahorias, tomate]","[varios bogavantes de a g., a uno más grande.]",1,0,[],0.7
16005,https://t1.rg.ltmcdn.com/es/images/2/5/1/img_marmitaco_de_atun_51152_600.jpg,Marmitaco de atún,https://www.recetasgratis.net/receta-de-marmitaco-de-atun-51152.html,[],NaN,0,0,0,0,0,principal,1,"[leche, huevos, jamón, aceite de oliva, sal, chorizo, guisantes, zanahorias, tomate]","[ajo, azafrán, perejil, aceite y sal.]",1,0,[],0.7
